# Renaming Files

In [1]:
import os

SRC_ROOT = "/kaggle/input/finaldataset"
DST_ROOT = "/kaggle/working/finaldataset"

rename_map = {
    "00 Anatomia Normal": "00_Normal",
    "01 Processos Inflamatórios Pulmonares (Pneumonia)": "01_Inflammation",
    "02 Maior Densidade (Derrame Pleural, Consolidação Atelectasica, Hidrotorax, Empiema)": "02_HighDensity",
    "03 Menor Densidade (Pneumotorax, Pneumomediastino, Pneumoperitonio)": "03_LowDensity",
    "04 Doenças Pulmonares Obstrutivas (Enfisema, Broncopneumonia, Bronquiectasia, Embolia)": "04_Obstructive",
    "05 Doenças Infecciosas Degenerativas (Tuberculose, Sarcoidose, Proteinose, Fibrose)": "05_Degenerative",
    "06 Lesões Encapsuladas (Abscessos, Nódulos, Cistos, Massas Tumorais, Metastases)": "06_Encapsulated",
    "07 Alterações de Mediastino (Pericardite, Malformações Arteriovenosas, Linfonodomegalias)": "07_Mediastinal",
    "08 Alterações do Tórax (Atelectasias, Malformações, Agenesia, Hipoplasias)": "08_ChestChanges",
}

os.makedirs(DST_ROOT, exist_ok=True)

for old_name, new_name in rename_map.items():
    src_path = os.path.join(SRC_ROOT, old_name)
    dst_path = os.path.join(DST_ROOT, new_name)

    if not os.path.isdir(src_path):
        print(f"Skipping (not found): {old_name}")
        continue

    if os.path.exists(dst_path):
        print(f"Already exists: {new_name}")
        continue

    print(f"Symlinking: {new_name} -> {old_name}")
    os.symlink(src_path, dst_path)

print("Done. Symlinked dataset is in:", DST_ROOT)


Symlinking: 00_Normal -> 00 Anatomia Normal
Symlinking: 01_Inflammation -> 01 Processos Inflamatórios Pulmonares (Pneumonia)
Symlinking: 02_HighDensity -> 02 Maior Densidade (Derrame Pleural, Consolidação Atelectasica, Hidrotorax, Empiema)
Symlinking: 03_LowDensity -> 03 Menor Densidade (Pneumotorax, Pneumomediastino, Pneumoperitonio)
Symlinking: 04_Obstructive -> 04 Doenças Pulmonares Obstrutivas (Enfisema, Broncopneumonia, Bronquiectasia, Embolia)
Symlinking: 05_Degenerative -> 05 Doenças Infecciosas Degenerativas (Tuberculose, Sarcoidose, Proteinose, Fibrose)
Symlinking: 06_Encapsulated -> 06 Lesões Encapsuladas (Abscessos, Nódulos, Cistos, Massas Tumorais, Metastases)
Symlinking: 07_Mediastinal -> 07 Alterações de Mediastino (Pericardite, Malformações Arteriovenosas, Linfonodomegalias)
Symlinking: 08_ChestChanges -> 08 Alterações do Tórax (Atelectasias, Malformações, Agenesia, Hipoplasias)
Done. Symlinked dataset is in: /kaggle/working/finaldataset


# Data Split

In [2]:
import os
import random
from pathlib import Path
import shutil

# ------------------------------
# CONFIG
# ------------------------------
# Source: symlinked, renamed dataset
data_dir = Path("/kaggle/working/finaldataset")

# Output: split dataset
output_dir = Path("/kaggle/working/splited")

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1
seed = 42

random.seed(seed)

# ------------------------------
# CLEAN OUTPUT DIR
# ------------------------------
if output_dir.exists():
    shutil.rmtree(output_dir)

# ------------------------------
# DISCOVER CLASSES
# ------------------------------
classes = sorted([d.name for d in data_dir.iterdir() if d.is_dir()])
print(f"Found classes: {classes}")

# ------------------------------
# SPLIT DATA
# ------------------------------
for cls in classes:
    class_dir = data_dir / cls
    images = sorted(class_dir.glob("*"))  # deterministic order
    random.shuffle(images)

    n_total = len(images)
    if n_total == 0:
        print(f"[{cls}] Skipping (empty folder)")
        continue

    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)
    n_test = n_total - n_train - n_val

    splits = {
        "train": images[:n_train],
        "val": images[n_train:n_train + n_val],
        "test": images[n_train + n_val:]
    }

    print(f"[{cls}] Total: {n_total} -> "
          f"Train: {n_train}, Val: {n_val}, Test: {n_test}")

    for split, files in splits.items():
        split_dir = output_dir / split / cls
        split_dir.mkdir(parents=True, exist_ok=True)

        for f in files:
            dst = split_dir / f.name
            if not dst.exists():
                os.symlink(f, dst)

print(f"\n✅ Splitting complete.")
print(f"📁 Data located at: {output_dir}")

Found classes: ['00_Normal', '01_Inflammation', '02_HighDensity', '03_LowDensity', '04_Obstructive', '05_Degenerative', '06_Encapsulated', '07_Mediastinal', '08_ChestChanges']
[00_Normal] Total: 1000 -> Train: 800, Val: 100, Test: 100
[01_Inflammation] Total: 1000 -> Train: 800, Val: 100, Test: 100
[02_HighDensity] Total: 1000 -> Train: 800, Val: 100, Test: 100
[03_LowDensity] Total: 1000 -> Train: 800, Val: 100, Test: 100
[04_Obstructive] Total: 1000 -> Train: 800, Val: 100, Test: 100
[05_Degenerative] Total: 1000 -> Train: 800, Val: 100, Test: 100
[06_Encapsulated] Total: 1000 -> Train: 800, Val: 100, Test: 100
[07_Mediastinal] Total: 1000 -> Train: 800, Val: 100, Test: 100
[08_ChestChanges] Total: 1000 -> Train: 800, Val: 100, Test: 100

✅ Splitting complete.
📁 Data located at: /kaggle/working/splited


# Data Load

In [32]:
import os
import tensorflow as tf
import numpy as np

base_dir   = '/kaggle/working/splited'
img_size   = (224, 224)
batch_size = 48 #32 pretrained
seed       = 42

tf.random.set_seed(seed)
np.random.seed(seed)

# Load datasets
train_ds_raw = tf.keras.utils.image_dataset_from_directory(
    os.path.join(base_dir, 'train'),
    label_mode='categorical',
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True,
    seed=seed
)

val_ds_raw = tf.keras.utils.image_dataset_from_directory(
    os.path.join(base_dir, 'val'),
    label_mode='categorical',
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    seed=seed
)

test_ds_raw = tf.keras.utils.image_dataset_from_directory(
    os.path.join(base_dir, 'test'),
    label_mode='categorical',
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    seed=seed
)

# Preprocessing function
def preprocess(image, label):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Apply preprocessing (no AUTOTUNE, no cache)
train_ds = train_ds_raw.map(preprocess)
val_ds   = val_ds_raw.map(preprocess)
test_ds  = test_ds_raw.map(preprocess)

print(f"Class names: {train_ds_raw.class_names}")
print(f"Number of classes: {len(train_ds_raw.class_names)}")

Found 7200 files belonging to 9 classes.
Found 900 files belonging to 9 classes.
Found 900 files belonging to 9 classes.
Class names: ['00_Normal', '01_Inflammation', '02_HighDensity', '03_LowDensity', '04_Obstructive', '05_Degenerative', '06_Encapsulated', '07_Mediastinal', '08_ChestChanges']
Number of classes: 9


# Helper funntions 

In [4]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np

def save_auc_roc(model, dataset, class_names, save_path="auc_roc.png"):
   
    # Get true labels and predictions
    y_true = np.concatenate([y for x, y in dataset], axis=0)
    y_pred = model.predict(dataset, verbose=0)
    
    n_classes = len(class_names)
    
    # Ensure y_true is one-hot encoded
    if y_true.shape[1] != n_classes:
        y_true = label_binarize(y_true, classes=range(n_classes))
    
    # Plot ROC for each class
    plt.figure(figsize=(10,8))
    
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_true[:, i], y_pred[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')
    
    # Plot diagonal line
    plt.plot([0,1], [0,1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multi-class ROC Curve')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.tight_layout()
    
    plt.savefig(save_path)
    plt.show()


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

def save_confusion_matrix(model, dataset, class_names, save_path="confusion_matrix.png"):
 
    # Get true labels and predictions
    y_true = np.concatenate([y for x, y in dataset], axis=0)
    y_pred_probs = model.predict(dataset, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true_labels = np.argmax(y_true, axis=1)
    
    # Compute confusion matrix
    cm = confusion_matrix(y_true_labels, y_pred)
    
    # Plot as heatmap
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.show()

In [21]:
import os
import matplotlib.pyplot as plt

def plot_training_curves(history, model_name="Model", save_path="."):

    # Ensure save directory exists
    os.makedirs(save_path, exist_ok=True)

    acc = history.history.get("accuracy")
    val_acc = history.history.get("val_accuracy")
    loss = history.history.get("loss")
    val_loss = history.history.get("val_loss")

    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(14, 5))

    # ---- Accuracy Plot ----
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, label="Train Accuracy")
    plt.plot(epochs, val_acc, label="Validation Accuracy")
    plt.title(f"{model_name} - Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)

    # ---- Loss Plot ----
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, label="Train Loss")
    plt.plot(epochs, val_loss, label="Validation Loss")
    plt.title(f"{model_name} - Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)

    # Save plot
    filename = f"{model_name}_training_curves.png"
    full_path = os.path.join(save_path, filename)
    plt.tight_layout()
    plt.savefig(full_path, dpi=300)
    plt.show()
    plt.close()

    print(f"✅ Plot saved at: {full_path}")


In [7]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import numpy as np

def save_classification_report(model, dataset, class_names, save_path="classification_report.png"):
 
    # Get true labels and predictions
    y_true = np.concatenate([y for x, y in dataset], axis=0)
    y_pred_probs = model.predict(dataset, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true_labels = np.argmax(y_true, axis=1)
    
    # Generate classification report
    report = classification_report(y_true_labels, y_pred, target_names=class_names, output_dict=True)
    
    # Convert report to text for display
    report_text = classification_report(y_true_labels, y_pred, target_names=class_names)
    
    # Plot report as figure
    plt.figure(figsize=(10, len(class_names)*0.6 + 2))
    plt.text(0, 1, "Classification Report", fontsize=14, fontweight='bold', va='top')
    plt.text(0, 0.95, report_text, fontsize=12, fontfamily='monospace', va='top')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.show()

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

def save_confusion_matrix(model, dataset, class_names, save_path="confusion_matrix.png"):
 
    # Get true labels and predictions
    y_true = np.concatenate([y for x, y in dataset], axis=0)
    y_pred_probs = model.predict(dataset, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true_labels = np.argmax(y_true, axis=1)
    
    # Compute confusion matrix
    cm = confusion_matrix(y_true_labels, y_pred)
    
    # Plot as heatmap
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.show()

In [9]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np

def save_auc_roc(model, dataset, class_names, save_path="auc_roc.png"):
   
    # Get true labels and predictions
    y_true = np.concatenate([y for x, y in dataset], axis=0)
    y_pred = model.predict(dataset, verbose=0)
    
    n_classes = len(class_names)
    
    # Ensure y_true is one-hot encoded
    if y_true.shape[1] != n_classes:
        y_true = label_binarize(y_true, classes=range(n_classes))
    
    # Plot ROC for each class
    plt.figure(figsize=(10,8))
    
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_true[:, i], y_pred[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')
    
    # Plot diagonal line
    plt.plot([0,1], [0,1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multi-class ROC Curve')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.tight_layout()
    
    plt.savefig(save_path)
    plt.show()

In [18]:
import numpy as np
from sklearn.metrics import roc_auc_score

def evaluate_model_with_auc(model, dataset, dataset_name="Dataset"):
    
    # 1. Standard model.evaluate() for loss and accuracy
    loss, accuracy = model.evaluate(dataset, verbose=0)

    # 2. Predict probabilities and get true labels
    y_true = []
    y_pred = []

    for x_batch, y_batch in dataset:
        preds = model.predict(x_batch, verbose=0)
        y_true.append(y_batch.numpy())
        y_pred.append(preds)

    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    # 3. Compute macro AUC for multi-class classification
    auc = roc_auc_score(y_true, y_pred, average='macro', multi_class='ovr')

    # 4. Print results
    print(f"{dataset_name} Loss: {loss:.4f}")
    print(f"{dataset_name} Accuracy: {accuracy:.4f}")
    print(f"{dataset_name} AUC (macro): {auc:.4f}")

    return {
        "loss": loss,
        "accuracy": accuracy,
        "auc_macro": auc
    }

# Custom CNN Arch. LXNet

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers


def conv_block(x, filters, k=3, s=1, name=None):
    x = layers.Conv2D(filters, k, strides=s, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("swish")(x)   
    return x


def build_lightxraynet(input_shape=(224,224,1), num_classes=9):
    inp = keras.Input(shape=input_shape)
    x = inp

    # Stem
    x = layers.Conv2D(32, 7, strides=2, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("swish", name="stem_swish")(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding="same", name="stem_pool")(x)
    x = layers.SpatialDropout2D(0.1)(x)
    
    # Block 1
    x = conv_block(x, 48, k=3, s=1, name="b1_1")
    x = conv_block(x, 48, k=3, s=1, name="b1_2")
    x = layers.MaxPooling2D(pool_size=2, strides=2, name="pool1")(x)  
    x = layers.SpatialDropout2D(0.05)(x)

    # Block 2
    x = conv_block(x, 72, k=3, s=1, name="b2_1")
    x = conv_block(x, 72, k=3, s=1, name="b2_2")
    x = layers.MaxPooling2D(pool_size=2, strides=2, name="pool2")(x)  
    x = layers.SpatialDropout2D(0.05)(x)

    # Block 3
    x = conv_block(x, 128, k=3, s=1, name="b3_1")
    x = conv_block(x, 128, k=3, s=1, name="b3_2")
    
    # Head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(num_classes, activation="softmax")(x)
    
    return keras.Model(inp, out)

# Build model with Swish
model = build_lightxraynet()

# Hyperparams Setup

In [ ]:
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model.compile(
    optimizer=Nadam(learning_rate=0.0003),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6, verbose=1),
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    callbacks=callbacks,
    epochs=40,
    verbose=1
)

In [ ]:
# Save model as .h5
modelName = "LXNet"
class_names = train_ds_raw.class_names
model.save("/kaggle/working/LightXRayNet.h5")
print("✅ Model saved as /kaggle/working/LightXRayNet.h5")

In [ ]:
val_results = evaluate_model_with_auc(model, val_ds, "Validation")
test_results = evaluate_model_with_auc(model, test_ds, "Test") 

In [ ]:
plot_training_curves(history, model_name=modelName, save_path="/kaggle/working")

In [ ]:
save_classification_report(model, test_ds, class_names, save_path="classification_report.png")

In [ ]:
save_confusion_matrix(model, test_ds, class_names, save_path="confusion_matrix.png")

In [ ]:
save_auc_roc(model, test_ds, class_names, save_path="auc_roc.png")


# Resnet

In [33]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


INPUT_SHAPE = (224, 224, 3)
NUM_CLASSES = 9

def resnet(
    input_shape=INPUT_SHAPE,
    num_classes=NUM_CLASSES,
    train_base=False,
    model_name="ResNet50V2"
):
    # Base model
    base_model = keras.applications.ResNet50V2(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape
    )
    base_model.trainable = train_base  # False for feature extraction

    # Model input
    inp = keras.Input(shape=input_shape)

    # Forward pass through base model
    x = base_model(inp, training=False)

    # Custom classification head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)

    # Output layer
    out = layers.Dense(num_classes, activation="softmax")(x)

    # Final model
    model = keras.Model(inputs=inp, outputs=out, name=model_name)

    return model

model = resnet()

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1),
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    callbacks=callbacks,
    epochs=3,
    verbose=1
)

In [ ]:

modelName = "ResNet50V2"
class_names = train_ds_raw.class_names
model.save("/kaggle/working/ResNet50V2.h5")
print("✅ Model saved as /kaggle/working/ResNet50V2.h5")

val_results = evaluate_model_with_auc(model, val_ds, "Validation")
test_results = evaluate_model_with_auc(model, test_ds, "Test") 
plot_training_curves(history, model_name=modelName, save_path="/kaggle/working")


save_classification_report(model, test_ds, class_names, save_path="classification_report.png")
save_confusion_matrix(model, test_ds, class_names, save_path="confusion_matrix.png")
save_auc_roc(model, test_ds, class_names, save_path="auc_roc.png")
